# Build a RAG agent with LangChain

## Overview

One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.

This class we will learn how to build a simple Q&A application over an unstructured text data source.

We will demonstrate:
- A RAG agent that executes searches with a simple tool. This is a good general-purpose implementation.
- A two-step RAG chain that uses just a single LLM call per query. This is a fast and effective method for simple queries.

## Concepts
We will cover the following concepts:
- **Indexing**: a pipeline for ingesting data from a source and indexing it. This usually happens in a separate process.
- **Retrieval** and **generation**: the actual RAG process, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

Once we’ve indexed our data, we will use an agent as our orchestration framework to implement the retrieval and generation steps.

In [ ]:
$ uv add langchain langchain-text-splitters langchain-community

In [ ]:
$ uv add "langchain[openai]"
# uv add "langchain[google-genai]"

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

# Components
We will need to select three components from LangChain’s suite of integrations.
1. Select a chat model:

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-4.1")

/Users/felipecisternas/Desktop/uc/practicos-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. Select an embeddings model:

In [ ]:
$ uv add langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

3. Select a vector store:

In [ ]:
$ uv add langchain-qdrant "langchain-core"

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

vector_size = len(embeddings.embed_query("sample text"))

if not client.collection_exists("test"):
    client.create_collection(
        collection_name="test",
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
    )
vector_store = QdrantVectorStore(
    client=client,
    collection_name="test",
    embedding=embeddings,
)

# 1. Indexing

Indexing commonly works as follows:
1. **Load**: First we need to load our data. This is done with Document Loaders.
2. **Split**: Text splitters break large Documents into smaller chunks. This is useful both for indexing data and passing it into a model, as large chunks are harder to search over and won’t fit in a model’s finite context window.
3. **Store**: We need somewhere to store and index our splits, so that they can be searched over later. This is often done using a VectorStore and Embeddings model.

## Loading documents
We need to first load the blog post contents. We can use `DocumentLoaders` for this, which are objects that load in data from a source and return a list of `Document` objects.
In this case we’ll use the `WebBaseLoader`, which uses urllib to load HTML from web URLs and BeautifulSoup to parse it to text.

We can customize the HTML -> text parsing by passing in parameters into the BeautifulSoup parser via bs_kwargs (see [BeautifulSoup docs](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)).

In this case only HTML tags with class “post-content”, “post-title”, or “post-header” are relevant, so we’ll remove all others.

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43047


In [ ]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


## Splitting documents

Our loaded document is over 42k characters which is too long to fit into the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we’ll split the Document into chunks for embedding and vector storage. This should help us retrieve only the most relevant parts of the blog post at run time.

As in the semantic search tutorial, we use a `RecursiveCharacterTextSplitter`, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


## Storing documents
Now we need to index our 66 text chunks so that we can search over them at runtime.

Our approach is to embed the contents of each document split and insert these embeddings into a vector store. Given an input query, we can then use vector search to retrieve relevant documents.

We can embed and store all of our document splits in a single command using the vector store and embeddings model selected at the start of the tutorial.

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['69e939f313224822abfbe25182c76049', '8078ae498a014d3da0a4d6654ef272eb', 'be3e00d1c57445c992e115ff8b2ceb03']


# 2. Retrieval and Generation
RAG applications commonly work as follows:
1. **Retrieve**: Given a user input, relevant splits are retrieved from storage using a Retriever.
2. **Generate**: A model produces an answer using a prompt that includes both the question with the retrieved data

Now let’s write the actual application logic.

We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

We will demonstrate:
- A RAG agent that executes searches with a simple tool. This is a good general-purpose implementation.
- A two-step RAG chain that uses just a single LLM call per query. This is a fast and effective method for simple queries.

## RAG agents
One formulation of a RAG application is as a simple agent with a tool that retrieves information. We can assemble a minimal RAG agent by implementing a tool that wraps our vector store:

In [ ]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

Here we use the `@[tool decorator][tool]` to configure the tool to attach raw documents as artifacts to each `ToolMessage`.

This will let us access document metadata in our application, separate from the stringified representation that is sent to the model.

Given our tool, we can construct the agent:

In [ ]:
from langchain.agents import create_agent

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

Let’s test this out. We construct a question that would typically require an iterative sequence of retrieval steps to answer:

In [ ]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

In [ ]:
for event in agent.stream(
    {
        "messages": [
            {"role": "user", "content": query}
        ]
    },
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_lI0BfjBTxIX6ONTlLiHm47N3)
 Call ID: call_lI0BfjBTxIX6ONTlLiHm47N3
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578, '_id': 'de63b1e592ff49b88e43cf0a642d1bd3', '_collection_name': 'test'}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct

Note that the agent:
1. Generates a query to search for a standard method for task decomposition.
2. Receiving the answer, generates a second query to search for common extensions of it.
3. Having received all necessary context, answers the question.